# Uitwerking Excerise 2
Student: Menno van Zandwijk <br /> Studentnummer: 1807647  <br />  Datum: 14-06-2022

## 1.1 Introductie

In dit notebook zijn de uitverkingen weergegeven van excerise twee waarbij een dataloader en data itereator zijn geprogrammeerd voor de EEG EYE dataset. De betreffende classes zijn weergegeven in de file 'data_tools' die is opgeslagen in de src-folder. In dit notebook zijn de classes beschreven, worden ze aangeroepen en de uitkomsten geanalyseerd. 

## 1.2 Importen functies en variabelen

In [1]:
import torch

Tensor = torch.Tensor
import sys
from pathlib import Path
from scipy.io import arff
import torch.nn.functional as F

sys.path.insert(0, "..")
from src.data import (
    data_tools,
)  # Importeren van functies en classes afkomstig van het bestand 'data_tools' die is opgeslagen in de src-folder.


2022-06-14 10:41:24.014000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-14 10:41:24.014033: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## 1.3 Data exploration

Middels onderstaande functie afkomstig uit het bestand 'data_tools' wordt de ruwe data geïmporteerd.

In [2]:
data = arff.loadarff(data_tools.get_eeg())


2022-06-14 10:41:35.442 | INFO     | src.data.data_tools:get_eeg:30 - Data is downloaded to ../data/raw/datasets/eeg_data.


In [3]:
print(
    f"De dataset bevat {len(data[0])} observaties met per observatie {len(data[0][1])} dimensies. De eerste veertien dimensies bevat registraties van hersenactiviteiten, de vijftiende observatie betreft het label of een oog tijdens de observatie open of dicht was."
)


De dataset bevat 14980 observaties met per observatie 15 dimensies. De eerste veertien dimensies bevat registraties van hersenactiviteiten, de vijftiende observatie betreft het label of een oog tijdens de observatie open of dicht was.


## 1.4 Data loader

Het doel van de dataloader is om de dataset op te delen in chunks waarbij het label (oog open of dicht) gelijk is. De dataset betreft een timeserie waardoor bij het 'chunken' van de data gekeken dient de worden naar de volgorde van de obervaties. Conform de opdracht dienen observaties met eenzelfde label toegevoegd te worden aan een chunk en dient een nieuwe chunk te worden aangemaakt zodra het label van een observatie wijzigt. Vervolgens dienen alle opvolgende observaties met het gewijzigde label weer verzameld te worden in een chunk waarna dit proces zich herhaalt totdat alle observaties zijn opgeslagen. Om de observaties te kunnen chunken is gebruik gemaakt van een for-loop die door de observaties itereert. De betreffende for-loop is hieronder weergegeven.

In [4]:
data = arff.loadarff(data_tools.get_eeg())
current_label = int(data[0][0][14])  # index 14 = label
EEG_batch = []  # Lege lijst waarin meerdere observaties worden opgeslagen
EEG_batches = []  # Lege lijst waarin meerdere batches in worden samengevoegd.
for record in data[0]:
    if int(record[14]) == current_label:
        EEG_values = (
            []
        )  # Lege lijst waarin de EEG_values van een bepaalde observatie in kunnen worden opgeslagen.
        for index, i in enumerate(record):
            if index != 14:
                EEG_values.append(i)
        EEG_values = torch.Tensor(EEG_values)
        EEG_batch.append(EEG_values)
    else:
        EEG_batch_label = (current_label, torch.stack(EEG_batch))
        EEG_batches.append(EEG_batch_label)
        current_label = int(record[14])
        EEG_batch = (
            []
        )  # Lege lijst waarin meerdere observaties in kunnen worden opgeslagen.
        EEG_values = (
            []
        )  # Lege lijst waarin de EEG_values van een bepaalde observatie in kunnen worden opgeslagen.
        for index, i in enumerate(record):
            if index != 14:
                EEG_values.append(i)
        EEG_values = torch.Tensor(EEG_values)
EEG_batch_label = (current_label, torch.stack(EEG_batch))
EEG_batches.append(EEG_batch_label)


2022-06-14 10:41:35.922 | INFO     | src.data.data_tools:get_eeg:30 - Data is downloaded to ../data/raw/datasets/eeg_data.


In [5]:
print(
    f"De for-loop heeft de {len(data[0])} observaties opgedeeld in {len(EEG_batches)} chunks."
)


De for-loop heeft de 14980 observaties opgedeeld in 24 chunks.


De hiervoor beschreven for-loop is vervolgens geïmplementeerd in de functie 'process_data' die in class 'EEGDataset' is opgenomen. De class 'EEGDataset' heeft daarbij de functies afkomstig van de class 'BaseDataset' inheritance. In onderstaande query is de ruwe dataset door de class gehaald om de 24 chunks te maken.

In [6]:
dataloader = data_tools.EEGDataset(datasetpath=data_tools.get_eeg())


2022-06-14 10:41:36.789 | INFO     | src.data.data_tools:get_eeg:30 - Data is downloaded to ../data/raw/datasets/eeg_data.


Middels de __len__ functie kan vervolgens de lengte (lees: aantal chunks) worden opgehaald.

In [7]:
dataloader.__len__()


24

Middels de __getitem__ functie kan vervolgens de data uit een specifieke chunk worden opgehaald. Hieruit blijkt dat de chunks tuples zijn met een label en tensor met meerdere observaties.

In [8]:
print(type(dataloader.__getitem__(0)))
dataloader.__getitem__(0)


<class 'tuple'>


(0,
 tensor([[4329.2300, 4009.2300, 4289.2300,  ..., 4280.5098, 4635.8999,
          4393.8501],
         [4324.6201, 4004.6201, 4293.8501,  ..., 4279.4902, 4632.8198,
          4384.1001],
         [4327.6899, 4006.6699, 4295.3799,  ..., 4282.0498, 4628.7202,
          4389.2300],
         ...,
         [4468.2100, 4044.6201, 4305.1299,  ..., 4367.6899, 4833.8501,
          4571.7900],
         [4461.0298, 4041.0300, 4300.0000,  ..., 4365.1299, 4826.6699,
          4558.4600],
         [4452.8198, 4032.3101, 4295.3799,  ..., 4353.3301, 4808.2100,
          4549.2300]]))

Middels de __shape__ blijkt dat de eerste chunk 188 observaties bevat met per observatie veertien dimensies.

In [9]:
dataloader[0][1].shape


torch.Size([188, 14])

## 1.5 Data iterater

Uit onderstaande query blijkt dat het aantal observaties per chunk verschilt en daardoor problemen kunnen ontstaan indien de chunks door een batch loop worden gehaald. Middels het windowen van de observaties is het mogelijk om gelijke batches te creëren die geschikt zijn om in te lezen in een model. In deze paragraaf zijn de navolgende twee manieren van windowing toegepast:
- Windowing zonder padding
- Windowing met padding 

In [10]:
for i in range(dataloader.__len__()):
    print(f"Chunk {i+1} heeft {len(dataloader[i][1])} observaties.")


Chunk 1 heeft 188 observaties.
Chunk 2 heeft 682 observaties.
Chunk 3 heeft 464 observaties.
Chunk 4 heeft 301 observaties.
Chunk 5 heeft 537 observaties.
Chunk 6 heeft 456 observaties.
Chunk 7 heeft 266 observaties.
Chunk 8 heeft 26 observaties.
Chunk 9 heeft 414 observaties.
Chunk 10 heeft 1009 observaties.
Chunk 11 heeft 891 observaties.
Chunk 12 heeft 683 observaties.
Chunk 13 heeft 724 observaties.
Chunk 14 heeft 2400 observaties.
Chunk 15 heeft 2050 observaties.
Chunk 16 heeft 970 observaties.
Chunk 17 heeft 651 observaties.
Chunk 18 heeft 42 observaties.
Chunk 19 heeft 204 observaties.
Chunk 20 heeft 51 observaties.
Chunk 21 heeft 1188 observaties.
Chunk 22 heeft 71 observaties.
Chunk 23 heeft 669 observaties.
Chunk 24 heeft 20 observaties.


### 1.5.1 Windowing zonder padding

Het doel van windowing is om op basis van de ongelijke chunks batches te creëren met een gelijke batch size. Middels onderstaande query worden de observaties in iedere chunk gewindowed met een window size van vijf. De betekent dat iedere batch vijf observaties bevat.

In [11]:
window_full = []
window_size = 5
for i in range(len(dataloader)):
    n_window = len(dataloader[i][1]) - window_size + 1
    time = torch.arange(0, window_size).reshape(1, -1)
    window = torch.arange(0, n_window).reshape(-1, 1)
    idx = time + window
    window_out = dataloader[i][1][idx]
    window_out_label = (dataloader[i][0], window_out)
    window_full.append(window_out_label)


De hiervoor beschreven window functie is vervolgens geïmplementeerd in de functie 'window' die in de class  'BaseDataIteratorWindowing' is opgenomen. 

In [12]:
dataloader_windowing = data_tools.BaseDataIteratorWindowing(dataloader, 5)


In [13]:
x, y = dataloader_windowing[0]
print(
    f"Door het windowen bevat de eerste chunk nu de volgende tensor: {y.shape}. Deze tensor bevat dus {len(dataloader_windowing[0][1])} batches met ieder {len(dataloader_windowing[0][1][1])} observaties en {len(dataloader_windowing[0][1][1][1])} dimensies."
)


Door het windowen bevat de eerste chunk nu de volgende tensor: torch.Size([184, 5, 14]). Deze tensor bevat dus 184 batches met ieder 5 observaties en 14 dimensies.


Middels onderstaande query is de werking van windowing goed zichtbaar. Het eerste window bevat de eerste vijf observaties uit de chunk:

In [14]:
print(dataloader[0][1][0:5])
dataloader_windowing[0][1][0]


tensor([[4329.2300, 4009.2300, 4289.2300, 4148.2100, 4350.2598, 4586.1499,
         4096.9199, 4641.0298, 4222.0498, 4238.4600, 4211.2798, 4280.5098,
         4635.8999, 4393.8501],
        [4324.6201, 4004.6201, 4293.8501, 4148.7202, 4342.0498, 4586.6699,
         4097.4399, 4638.9702, 4210.7700, 4226.6699, 4207.6899, 4279.4902,
         4632.8198, 4384.1001],
        [4327.6899, 4006.6699, 4295.3799, 4156.4102, 4336.9199, 4583.5898,
         4096.9199, 4630.2598, 4207.6899, 4222.0498, 4206.6699, 4282.0498,
         4628.7202, 4389.2300],
        [4328.7202, 4011.7900, 4296.4102, 4155.8999, 4343.5898, 4582.5601,
         4097.4399, 4630.7700, 4217.4399, 4235.3799, 4210.7700, 4287.6899,
         4632.3101, 4396.4102],
        [4326.1499, 4011.7900, 4292.3101, 4151.2798, 4347.6899, 4586.6699,
         4095.8999, 4627.6899, 4210.7700, 4244.1001, 4212.8198, 4288.2100,
         4632.8198, 4398.4600]])


tensor([[4329.2300, 4009.2300, 4289.2300, 4148.2100, 4350.2598, 4586.1499,
         4096.9199, 4641.0298, 4222.0498, 4238.4600, 4211.2798, 4280.5098,
         4635.8999, 4393.8501],
        [4324.6201, 4004.6201, 4293.8501, 4148.7202, 4342.0498, 4586.6699,
         4097.4399, 4638.9702, 4210.7700, 4226.6699, 4207.6899, 4279.4902,
         4632.8198, 4384.1001],
        [4327.6899, 4006.6699, 4295.3799, 4156.4102, 4336.9199, 4583.5898,
         4096.9199, 4630.2598, 4207.6899, 4222.0498, 4206.6699, 4282.0498,
         4628.7202, 4389.2300],
        [4328.7202, 4011.7900, 4296.4102, 4155.8999, 4343.5898, 4582.5601,
         4097.4399, 4630.7700, 4217.4399, 4235.3799, 4210.7700, 4287.6899,
         4632.3101, 4396.4102],
        [4326.1499, 4011.7900, 4292.3101, 4151.2798, 4347.6899, 4586.6699,
         4095.8999, 4627.6899, 4210.7700, 4244.1001, 4212.8198, 4288.2100,
         4632.8198, 4398.4600]])

Het tweede window bevat de observaties twee t/m zes zoals hieronder weergeven.  De window schuift namelijk steeds een observaties op tot het einde.  

In [15]:
print(dataloader[0][1][0:6])
dataloader_windowing[0][1][1]


tensor([[4329.2300, 4009.2300, 4289.2300, 4148.2100, 4350.2598, 4586.1499,
         4096.9199, 4641.0298, 4222.0498, 4238.4600, 4211.2798, 4280.5098,
         4635.8999, 4393.8501],
        [4324.6201, 4004.6201, 4293.8501, 4148.7202, 4342.0498, 4586.6699,
         4097.4399, 4638.9702, 4210.7700, 4226.6699, 4207.6899, 4279.4902,
         4632.8198, 4384.1001],
        [4327.6899, 4006.6699, 4295.3799, 4156.4102, 4336.9199, 4583.5898,
         4096.9199, 4630.2598, 4207.6899, 4222.0498, 4206.6699, 4282.0498,
         4628.7202, 4389.2300],
        [4328.7202, 4011.7900, 4296.4102, 4155.8999, 4343.5898, 4582.5601,
         4097.4399, 4630.7700, 4217.4399, 4235.3799, 4210.7700, 4287.6899,
         4632.3101, 4396.4102],
        [4326.1499, 4011.7900, 4292.3101, 4151.2798, 4347.6899, 4586.6699,
         4095.8999, 4627.6899, 4210.7700, 4244.1001, 4212.8198, 4288.2100,
         4632.8198, 4398.4600],
        [4321.0298, 4004.6201, 4284.1001, 4153.3301, 4345.6401, 4587.1802,
         4093.3

tensor([[4324.6201, 4004.6201, 4293.8501, 4148.7202, 4342.0498, 4586.6699,
         4097.4399, 4638.9702, 4210.7700, 4226.6699, 4207.6899, 4279.4902,
         4632.8198, 4384.1001],
        [4327.6899, 4006.6699, 4295.3799, 4156.4102, 4336.9199, 4583.5898,
         4096.9199, 4630.2598, 4207.6899, 4222.0498, 4206.6699, 4282.0498,
         4628.7202, 4389.2300],
        [4328.7202, 4011.7900, 4296.4102, 4155.8999, 4343.5898, 4582.5601,
         4097.4399, 4630.7700, 4217.4399, 4235.3799, 4210.7700, 4287.6899,
         4632.3101, 4396.4102],
        [4326.1499, 4011.7900, 4292.3101, 4151.2798, 4347.6899, 4586.6699,
         4095.8999, 4627.6899, 4210.7700, 4244.1001, 4212.8198, 4288.2100,
         4632.8198, 4398.4600],
        [4321.0298, 4004.6201, 4284.1001, 4153.3301, 4345.6401, 4587.1802,
         4093.3301, 4616.9199, 4202.5601, 4232.8198, 4209.7402, 4281.0298,
         4628.2100, 4389.7402]])

In [16]:
print(
    f"De eerste chunk van de dataloader bevat {len(dataloader[0][1])} observaties, de eerste chunk na windowing maar {len(window_full[0][1])} windows. Dit verschil wordt veroorzaakt doordat aan het eind van de eerste chunck niet voldoende observaties beschikbaar zijn om nog volledige windows te creëren."
)


De eerste chunk van de dataloader bevat 188 observaties, de eerste chunk na windowing maar 184 windows. Dit verschil wordt veroorzaakt doordat aan het eind van de eerste chunck niet voldoende observaties beschikbaar zijn om nog volledige windows te creëren.


### 1.5.2 Windowing met padding


Zoals weergegeven in paragraaf 1.5 varieert het aantal observaties per chunk van 20 observaties (chunk 24) tot 2.400 observaties (chunk 14). Indien bij het windowen wordt gekozen voor een window size die groter is dan het aantal observaties in een chunk, ontstaat een error. Om deze error te voorkomen kan middels padding de lengte van een chunk worden vergroot met 0-waarden zodat de gekozen window size wel kan worden toegepast. In deze paragraaf is toegelicht op welke wijze de padding is toegevoegd aan de chunks.

Indien tijden het windowen wordt gekozen voor een te grote window size, ontstaat een error:

In [17]:
dataloader_windowing = data_tools.BaseDataIteratorWindowing(dataloader, 30)


RuntimeError: upper bound and larger bound inconsistent with step sign

Om deze error te voorkomen is ontstaande for-loop geprogrammeerd die, indien de window size groter is dan het aantal observaties in een chunk, padding toegevoegt. In onderstaande for-loop is een window size ingegeven van 30 observaties: 

In [19]:
data = dataloader
window_size = 30
list_padded = []
for i in range(len(data)):
    len_chunck = len(data[i][1])
    padding_value = window_size - len_chunck
    if padding_value > 0:
        chuck_padding = F.pad(
            input=data[i][1], pad=(0, 0, 0, padding_value), mode="constant", value=0
        )
        padding_label = (data[i][0], chuck_padding)
        list_padded.append(padding_label)
    else:
        chunck = (data[i][0], data[i][1])
        list_padded.append(chunck)


In [20]:
print(
    f"Chunk 23 bevat zonder padding {len(dataloader[23][1])} observaties, op basis van een window size van {window_size} bevat chunk 23 na padding {len(list_padded[23][1])} observaties. De toename in het aantal observaties is veroorzaakt doordat padding is toegevoegd aan de betreffende chunk."
)


Chunk 23 bevat zonder padding 20 observaties, op basis van een window size van 30 bevat chunk 23 na padding 30 observaties. De toename in het aantal observaties is veroorzaakt doordat padding is toegevoegd aan de betreffende chunk.


Middels de hiervoor beschreven functie zijn er tijdens het windowen geen beperkingen meer aanwezig voor wat betreft batch size. De betreffende for-loop is middels een functie geïmplementeerd in de class 'BaseDataIteratorPaddingWindowing' waarin eerst padding wordt toegepast alvorens de window functie wordt aangeroepen. In onderstaande query wordt de betreffende class aangeroepen. 

In [21]:
dataloader_padding_windowing = data_tools.BaseDataIteratorPaddingWindowing(
    dataloader, 30
)


In [22]:
x, y = dataloader_padding_windowing[0]
print(
    f"Door de het windowen bevat de eerste chunk nu de volgende tensor: {y.shape}. Deze tensor bevat dus {len(dataloader_padding_windowing[0][1])} batches met ieder {len(dataloader_padding_windowing[0][1][0])} observaties en {len(dataloader_padding_windowing[0][1][0][0])} dimensies."
)


Door de het windowen bevat de eerste chunk nu de volgende tensor: torch.Size([159, 30, 14]). Deze tensor bevat dus 159 batches met ieder 30 observaties en 14 dimensies.
